In [1]:
import numpy as np 
import seaborn as sns
import pandas as pd
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn import svm
import statsmodels.formula.api as smf
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/barankilic/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/07/73n4qj9d0fg68hxg63thmpd00000gp/T/ipykernel_74138/1067538934.py", line 1, in <module>
    import numpy as np
  File "/Users/barankilic/opt/anaconda3/lib/python3.9/site-packages/numpy/__init__.py", line 327, in <module>
    __mkl_version__ = "{MajorVersion}.{MinorVersion}.{UpdateVersion}".format(**mkl.get_version())
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/barankilic/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Tra

TypeError: object of type 'NoneType' has no len()

In [ ]:
#Get Data
dataset = pd.read_csv('desktop/v_shipment.csv')
dataset

In [ ]:
dataset = dataset[dataset.invoice_company_title != 'TRENDYOL LOJISTIK A.Ş']
dataset

In [ ]:
# Get Delivered Shipments
accepted = ('APPROVED', 'IN_TRANSIT', 'IN_DELIVERY', 'IN_CROSSDOCK', 'DELIVERED', 'IN_PICKUP')

dataset = dataset[dataset.latest_status.isin(accepted)]
dataset

In [ ]:
#Get Spot Prices
price_type = ['SPOT']
#dataset = dataset[dataset.price_type.isin(price_type)]
dataset

In [ ]:
# GET pickup and dropoffs
routes = ('İstanbul_İstanbul', 'İstanbul_İzmir', 'İstanbul_Ankara', 'İstanbul_Kocaeli', 'İstanbul_Bursa', 
          'İstanbul_Antalya', 
          'Kocaeli_İstanbul', 'İstanbul_Adana','İstanbul_Tekirdağ','İstanbul_Gaziantep','İstanbul_Balıkesir',
          'İstanbul_Eskişehir','İstanbul_Muğla','Kocaeli_İzmir','İstanbul_Mersin','İstanbul_Sakarya',
          'İstanbul_Samsun','Kocaeli_Ankara','İstanbul_Denizli','İstanbul_Kayseri')
dataset = dataset[dataset.routes.isin(routes)]
dataset

In [ ]:
# Get Applicable Columns
data = pd.concat([dataset['latest_status'],dataset['totalkgds'],dataset['pickup_city'],dataset['dropoff_city'] ,dataset['pickup_address_type'], dataset['dropoff_address_type'],
                  dataset['category'], dataset['price_type'], 
                  dataset['shipment_type'], dataset['vehicle_type'],dataset['vehicle_body_type'],
                  dataset['vehicle_package_type'], dataset['vehicle_way_of_loading'],
                  dataset['vehicle_tonnage_per_vehicle'],dataset['payment_option'],
                  dataset['invoice_type'], dataset['tenant_signup_flow'], 
                  dataset['total_cost_amount_in_tl'], dataset['usd_tl'],
                  dataset['price_base_amount_in_tl']
                 ,dataset['routes']], axis = 1)
data

In [ ]:
#Drop Price type and Latest Status
data = data.drop(columns= 'price_type')


data

In [ ]:
#Split Data by Shipment Type
data_ftl = data[data.shipment_type == 'FTL']

In [ ]:
data_ftl = data_ftl.drop(columns = 'shipment_type')
data_ftl = data_ftl.drop(columns = 'latest_status')
data_ftl = data_ftl.drop(columns = 'price_base_amount_in_tl')
data_ftl = data_ftl.drop(columns = 'payment_option')
data_ftl = data_ftl.drop(columns = 'tenant_signup_flow')
data_ftl = data_ftl.drop(columns = 'invoice_type')


In [ ]:
labelencoder = LabelEncoder()
data_ftl['category'] = labelencoder.fit_transform(data_ftl['category'])
data_ftl['vehicle_type'] = labelencoder.fit_transform(data_ftl['vehicle_type'])
data_ftl['vehicle_body_type'] = labelencoder.fit_transform(data_ftl['vehicle_body_type'])
data_ftl['vehicle_package_type'] = labelencoder.fit_transform(data_ftl['vehicle_package_type'])
data_ftl['vehicle_way_of_loading'] = labelencoder.fit_transform(data_ftl['vehicle_way_of_loading'])
data_ftl['pickup_address_type'] = labelencoder.fit_transform(data_ftl['pickup_address_type'])
data_ftl['dropoff_address_type'] = labelencoder.fit_transform(data_ftl['dropoff_address_type'])
data_ftl['routes'] = labelencoder.fit_transform(data_ftl['routes'])
data_ftl['pickup_city'] = labelencoder.fit_transform(data_ftl['pickup_city'])
data_ftl['dropoff_city'] = labelencoder.fit_transform(data_ftl['dropoff_city'])


data_ftl.info()

In [ ]:
data_ftl = data_ftl.dropna()
data_ftl.info()

In [ ]:
data_ftl.totalkgds = data_ftl.totalkgds.astype(int)
data_ftl.vehicle_tonnage_per_vehicle = data_ftl.vehicle_tonnage_per_vehicle.astype(int)
data_ftl.total_cost_amount_in_tl = data_ftl.total_cost_amount_in_tl.astype(int)
data_ftl.usd_tl = data_ftl.usd_tl.astype(int)

data_ftl.info()

In [ ]:
X_train_ftl, X_test_ftl, y_train_ftl, y_test_ftl = train_test_split(data_ftl.drop(columns = 'total_cost_amount_in_tl'), data_ftl.total_cost_amount_in_tl, test_size=0.2, random_state=0)
X_train_ftl.shape, X_test_ftl.shape

In [ ]:
cls = svm.SVC(kernel = 'linear')

In [ ]:
cls.fit(X_train_ftl,y_train_ftl)

In [ ]:
prediction = cls.predict(X_test_ftl)

import pathlib
pathlib.Path(svm).parent.resolve()

NameError: name 'svm' is not defined